In [ ]:
import gym
import random
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)

from scripts.reinforce_PPORLHF import reinforce_rwd2go_PPO_RLHF
from scripts.utils import pref_load

from scripts.model import getPolicy
import torch
import torch.nn.functional as F

from scripts.reinforce_PPORLHF import reinforce_rwd2go_PPO_RLHF

%load_ext autoreload
%autoreload 2
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
SEED=0
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

ENV_NAME = 'CartPole-v0'

env = gym.make(ENV_NAME)

K = 500

POLICY1_NAME = "policies/policy1.pth"
POLICY2_NAME = "policies/policy2_with_both.pth"
pref_data = pref_load(f"pref_data/pref_data_{K}_{ENV_NAME}.pickle")

In [ ]:
class RewardModel(nn.Module):
    def __init__(self, state_size=4, action_size=1, hidden_size=32):
        super(RewardModel, self).__init__()
        self.fc1 = nn.Linear(state_size + action_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        return x

    def predict_reward(self, state, action):
        state = state
        action = torch.tensor(action).reshape(1,1)  
        
        state_action = torch.cat((state, action), dim=1)
        reward = self.forward(state_action).cpu()
        return reward
    

lr        = 3e-3
epochs    = 20

reward_model = RewardModel(state_size=4, action_size=1)

optimizer = torch.optim.Adam(reward_model.parameters(), lr=lr)

def trajectory_reward(reward_model, states, actions):
    total_reward = torch.tensor(0., device=device)
    for s, a in zip(states, actions):
        s_t = torch.tensor(s, dtype=torch.float32, device=device)
        reward = reward_model.predict_reward(s_t.unsqueeze(0), a).squeeze(0)
        total_reward += reward.squeeze(0)
    return total_reward

losses_reward_model = []
for epoch in range(1, epochs+1):
    total_loss = 0.0
    
    for s0, tau_plus, tau_minus in pref_data:

        reward_plus = trajectory_reward(reward_model, tau_plus["states"], tau_plus["actions"])
        reward_minus = trajectory_reward(reward_model, tau_minus["states"], tau_minus["actions"])
        total_loss += - torch.log(torch.exp(reward_plus) / (torch.exp(reward_minus) + torch.exp(reward_plus)))

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    avg_loss = total_loss / len(pref_data)
    losses_reward_model.append(avg_loss.detach().numpy().item())
    
    print(f"Epoch {epoch}/{epochs} — avg loss: {avg_loss:.4f}")

plt.plot(np.arange(epochs), losses_reward_model)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training loss reward model RLHF")
plt.legend()
plt.savefig(f"RLHF_{ENV_NAME}_{K}.jpg")
plt.show()

In [ ]:
policy2 = getPolicy(ENV_NAME=ENV_NAME).to(device)
policy2.load_state_dict(torch.load(POLICY2_NAME))


opt1 = optim.Adam(policy2.parameters(), lr=1e-3)
reward_model.eval()

reward_evaluation_every=10
losses, mean_returns, std_returns = reinforce_rwd2go_PPO_RLHF(env, policy2, opt1, reward_model, n_episodes=2000, reward_evaluation_every=reward_evaluation_every);

x = np.arange(len(losses)) * reward_evaluation_every
plt.plot(x, mean_returns, color="tab:blue", label="Mean Reward")
plt.fill_between(x, mean_returns - std_returns, mean_returns + std_returns, color="tab:blue", alpha=0.3)
plt.ylabel("Mean Reward")
plt.xlabel("Episodes")

plt.tight_layout()
plt.title("Training Progress")
plt.savefig(f"Mean_reward_RLHF_{ENV_NAME}_{K}.jpg")
plt.show()